In [1]:
import os
import cv2
import numpy as np
from datetime import datetime

from keras_contrib.layers.normalization.instancenormalization import InstanceNormalization
from keras_contrib.layers import GroupNormalization
from keras.layers import Input, Dense, Reshape, Flatten, Dropout, Concatenate, Add
from keras.layers import BatchNormalization, Activation, MaxPooling2D, Conv2DTranspose
from keras.layers.advanced_activations import LeakyReLU
from keras.layers.convolutional import UpSampling2D, Conv2D
from keras.models import Sequential, Model, model_from_json
from keras.optimizers import Adam
from keras.initializers import RandomNormal
import sys

Using TensorFlow backend.


In [2]:
CHECKPOINT = "checkpoint/"
SAVED_IMAGES_PATH = "saved_images/"
LOG_PATH = "checkpoint/log.txt"

EPOCHS = 1000
BATCH_SIZE = 1

# Saves Model in every N minutes
TIME_INTERVALS = 2 

# Show Summary of Models
SHOW_SUMMARY = True

# Learning rate of Generator and Discriminator
DISCRIMINATOR_LR_RATE = 0.0002
GENERATOR_LR_RATE = 0.0002

In [3]:
# Input Image Dimensions
IMAGE_ROWS = 256
IMAGE_COLS = 256
IMAGE_CHANNELS = 3
IMAGE_SHAPE = (IMAGE_ROWS, IMAGE_COLS, IMAGE_CHANNELS)

In [4]:
from dataloader import Data
data = Data()

### DISCRIMINIATOR (PATHGAN)

In [5]:
d_layers = 3 # discriminator size
d_filter_size = 64 # filter size of first layer
d_dropout = 0.4
d_loss = 'mse'
d_optimizer = Adam(DISCRIMINATOR_LR_RATE, 0.5)

Instructions for updating:
Colocations handled automatically by placer.


In [6]:
# Calculate output shape of D (PatchGAN)
patch = int(IMAGE_ROWS / 2**(d_layers))
patch = (patch, patch, 1)
print("Patch shape: ", patch)

Patch shape:  (32, 32, 1)


In [7]:
def build_d_layer(layer_input, filters, f_size=4, strides =2, normalization=True, dropout_rate=d_dropout):
    
    d = Conv2D(filters, kernel_size=f_size, strides=strides, padding='same')(layer_input)
    d = LeakyReLU(alpha=0.2)(d)
    if dropout_rate:
        d = Dropout(dropout_rate)(d)
    if normalization:
        d = InstanceNormalization()(d)
    return d
    
    
def build_discriminator():
    image = Input(shape=IMAGE_SHAPE)
    layers  = []
    
    # Making  N (d_layers) Layers 
    for i in range(d_layers):
        filter_size = d_filter_size * (2 ** i)
        if not i:
            layer = build_d_layer(image, filter_size, normalization=False)
        else:
            layer = build_d_layer(layers[-1], filter_size)
            
        layers.append(layer)
    layers.append(layer)    
    
    confidence = Conv2D(1, kernel_size=4, strides=1,activation='sigmoid', padding='same')(layers[-1])
    
    # Model(input, output)
    return Model(image, confidence)

In [8]:
# Discriminator Initialization
DCRM_A = build_discriminator()
DCRM_B = build_discriminator()

# Compile the Discriminator Model
DCRM_A.compile(loss=d_loss, optimizer=d_optimizer, metrics=['accuracy'])
DCRM_B.compile(loss=d_loss, optimizer=d_optimizer, metrics=['accuracy'])

# Show Summary
if SHOW_SUMMARY:
    print('Summary DCRM_A:')
    DCRM_A.summary()
    print('Summary DCRM_B:')
    DCRM_B.summary()

Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Summary DCRM_A:
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 256, 256, 3)       0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 128, 128, 64)      3136      
_________________________________________________________________
leaky_re_lu_1 (LeakyReLU)    (None, 128, 128, 64)      0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 128, 128, 64)      0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 64, 64, 128)       131200    
_________________________________________________________________
leaky_re_lu_2 (LeakyReLU)    (None, 64, 64, 128)       0         
___________

### GENERATOR (U-Net)

In [ ]:
g_filter_size = 64 # filter size of first layer
g_dropout = 0.2
dropout = 0.25
g_optimizer = Adam(GENERATOR_LR_RATE, 0.5)

In [ ]:
def build_conv2d(layer_input, filters, strides=2, f_size=4):
    g = Conv2D(filters, kernel_size=f_size, strides=strides, padding='same')(layer_input)
    g = LeakyReLU(alpha=0.2)(g)
    g = InstanceNormalization()(g)
    return g


def build_deconv2d(layer_input, skip_input, filters, strides=2, f_size=4, dropout_rate=g_dropout):
    g = Conv2DTranspose(filters, kernel_size=f_size, strides=strides, padding='same')(layer_input)
    g = LeakyReLU(alpha=0.2)(g)
    if dropout_rate:
        g = Dropout(dropout_rate)(g)
    g = InstanceNormalization()(g)
    g = Concatenate()([g, skip_input])
    return g


def build_generator():
    image = Input(shape=IMAGE_SHAPE)
    layers = []
    
    #  DownSampling the layers
    conv1 = build_conv2d(image, g_filter_size)
    conv2 = build_conv2d(conv1, g_filter_size*2)
    conv3 = build_conv2d(conv2, g_filter_size*4)
    conv4 = build_conv2d(conv3, g_filter_size*8)
    conv5 = build_conv2d(conv4, g_filter_size*16)
    
    # UpSampling the Layers
    deconv1 = build_deconv2d(conv5, conv4, g_filter_size*8)
    deconv2 = build_deconv2d(deconv1, conv3, g_filter_size*4)
    deconv3 = build_deconv2d(deconv2, conv2, g_filter_size*2)
    deconv4 = build_deconv2d(deconv3, conv1, g_filter_size)
    deconv5 = UpSampling2D(size=2)(deconv4)
    
    generated_image = Conv2D(IMAGE_CHANNELS, kernel_size=4, strides=1, padding='same', activation='tanh')(deconv5)
    
    return Model(image, generated_image)

In [ ]:
# Generator Initialization
GEN_AB = build_generator()
GEN_BA = build_generator()

# Generator input PlaceHolder
IMG_A = Input(shape=IMAGE_SHAPE)
IMG_B = Input(shape=IMAGE_SHAPE)

# Generator Summary
if SHOW_SUMMARY:
    print('GEN_AB')
    GEN_AB.summary()
    print('GEN_BA')
    GEN_BA.summary()

GEN_AB
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            (None, 256, 256, 3)  0                                            
__________________________________________________________________________________________________
conv2d_9 (Conv2D)               (None, 128, 128, 64) 3136        input_3[0][0]                    
__________________________________________________________________________________________________
leaky_re_lu_7 (LeakyReLU)       (None, 128, 128, 64) 0           conv2d_9[0][0]                   
__________________________________________________________________________________________________
instance_normalization_5 (Insta (None, 128, 128, 64) 2           leaky_re_lu_7[0][0]              
__________________________________________________________________________________________________
con

### COMBINE NETWORK - CYCLEGAN (ADVERSARIAL NET)

In [ ]:
# Prepare fake images from Generator
FAKE_IMG_A = GEN_BA(IMG_B)
FAKE_IMG_B = GEN_AB(IMG_A)

# Reconstruct fake images back to original
RECONSTRUCT_A = GEN_BA(FAKE_IMG_B)
RECONSTRUCT_B = GEN_AB(FAKE_IMG_A)

# Original Idendity of the Image
ID_A = GEN_BA(IMG_A)
ID_B = GEN_AB(IMG_B)

# Discriminator Model shouldn't be affected during Adverserial(Combined Model) Optimization
# So the discriminator model is frozen 
DCRM_A.trainable = False
DCRM_B.trainable = False

# Discriminator Confidence of Fake images
CONF_FAKE_IMG_A = DCRM_A(FAKE_IMG_A)
CONF_FAKE_IMG_B = DCRM_B(FAKE_IMG_B)

### INITIALIZE ADVERSIAL NET - COMBINED NETWORK

In [ ]:
'''
The Combined model will calculate all this loss:
1) Discriminator loss of generated image
2) Reconstruction loss from generated image back to the original image
3) Identity loss from original image to generated image

This loss will be used to optimize the Generator Model
'''
COMBINED = Model([IMG_A, IMG_B],
                [CONF_FAKE_IMG_A, CONF_FAKE_IMG_B,
                 RECONSTRUCT_A, RECONSTRUCT_B,
                 ID_A, ID_B])

COMBINED.compile(loss= ['mse', 'mse',
                        'mae', 'mae',
                        'mae', 'mae'], 
                 loss_weights = [1, 1,
                                 10.0, 10.0,
                                 1.0, 1.0],
                 optimizer = g_optimizer)

### Utilities
    - Saving Model
    - Loading Model
    - Saving Logs
    - Save Image

In [ ]:
# Saving Discriminator and Generator Models at Checkpoint Path
def save_model():
    global DCRM_A, DCRM_B, GEN_AB, GEN_BA
    models = [DCRM_A, DCRM_B, GEN_AB, GEN_BA]
    model_names = ['DCRM_A', 'DCRM_B', 'GEN_AB', 'GEN_BA']

    for model, model_name in zip(models, model_names):
        model_path =  CHECKPOINT + "%s.json" % model_name
        weights_path = CHECKPOINT + "/%s.hdf5" % model_name
        options = {"file_arch": model_path, 
                    "file_weight": weights_path}
        json_string = model.to_json()
        open(options['file_arch'], 'w').write(json_string)
        model.save_weights(options['file_weight'])
    print("Saved Model")

In [ ]:
# Loading Discriminator and Generator Models from CHECKPOINT Path
def load_model():
    # Checking if all the model exists
    model_names = ['DCRM_A', 'DCRM_B', 'GEN_AB', 'GEN_BA']
    files = os.listdir(CHECKPOINT)
    for model_name in model_names:
        if model_name+".json" not in files or\
           model_name+".hdf5" not in files:
            print("Models not Found")
            return
    
    global DCRM_A, DCRM_B, GEN_AB, GEN_BA, d_optimizer, d_loss
    optimizer = Adam(0.0002, 0.5)
    
    # load DCRM_A Model
    model_path = CHECKPOINT + "%s.json" % 'DCRM_A'
    weight_path = CHECKPOINT + "%s.hdf5" % 'DCRM_A'
    with open(model_path, 'r') as f:
        DCRM_A = model_from_json(f.read(), {'InstanceNormalization': InstanceNormalization})
    DCRM_A.load_weights(weight_path)
    DCRM_A.compile(loss=d_loss, optimizer=d_optimizer, metrics=['accuracy'])
    
    # load DCRM_B Model
    model_path = CHECKPOINT + "%s.json" % 'DCRM_B'
    weight_path = CHECKPOINT + "%s.hdf5" % 'DCRM_B'
    with open(model_path, 'r') as f:
        DCRM_B = model_from_json(f.read(), {'InstanceNormalization': InstanceNormalization})
    DCRM_B.load_weights(weight_path)
    DCRM_B.compile(loss=d_loss, optimizer=d_optimizer, metrics=['accuracy'])
    
    # # load GEN_AB Model
    model_path = CHECKPOINT + "%s.json" % 'GEN_AB'
    weight_path = CHECKPOINT + "%s.hdf5" % 'GEN_AB' 
    with open(model_path, 'r') as f:
        GEN_AB = model_from_json(f.read(), {'InstanceNormalization': InstanceNormalization})
    GEN_AB.load_weights(weight_path)

    #load GEN_BA Model
    model_path = CHECKPOINT + "%s.json" % 'GEN_BA'
    weight_path = CHECKPOINT + "%s.hdf5" % 'GEN_BA'
    with open(model_path, 'r') as f:
         GEN_BA = model_from_json(f.read(), {'InstanceNormalization': InstanceNormalization})
    GEN_BA.load_weights(weight_path)
        
    print("Loaded Model")

In [ ]:
# Saving log in LOG_PATH
def save_log(log):
    with open(LOG_PATH, 'a') as f:
        f.write("%s\n" %log)

In [ ]:
# Save images to SAVE_IMAGE directory
#          #################################
# img =    #IMG_A, FAKE_A, RECONSTRUCTED_A#
#          #IMG_B, FAKE_B, RECONSTRUCTED_B#
           ################################

def save_image(epoch, steps):
    TEST_DATA_A, TEST_DATA_B = data.get_data(1)
    # If TEST_DATA is None, data exhausted
    # Reinitialise TEST_DATA
    if TEST_DATA_A is None or TEST_DATA_B is None:
        TEST_DATA_A, TEST_DATA_B = data.get_data(1)
    
    for i in range(TEST_DATA_A.shape[0]):
        original_A = TEST_DATA_A[i]
        original_B = TEST_DATA_B[i]
        real_A = original_A / 127.5 - 1
        real_B = original_B / 127.5 - 1
        real_A = np.reshape(real_A, [1, real_A.shape[0], real_A.shape[1], real_A.shape[2]])
        real_B = np.reshape(real_B, [1, real_B.shape[0], real_B.shape[1], real_B.shape[2]])
        
        fake_A = GEN_BA.predict(real_B)
        fake_A = (fake_A + 1) * 127.5
        fake_A = fake_A.astype(np.uint8)
        fake_B = GEN_AB.predict(real_A)
        fake_B = (fake_B + 1) * 127.5
        fake_B = fake_B.astype(np.uint8)
        
        recon_A = GEN_BA.predict(fake_B)
        recon_A = (recon_A + 1) * 127.5
        recon_A = recon_A.astype(np.uint8)
        recon_B = GEN_AB.predict(fake_A)
        recon_B = (recon_B + 1) * 127.5
        recon_B = recon_B.astype(np.uint8)
        
        image_1 = np.concatenate((original_A, fake_B[0], recon_A[0]), axis=1)
        image_2 = np.concatenate((original_B, fake_A[0], recon_B[0]), axis=1)
        image = np.concatenate((image_1, image_2), axis=0)
        image_path = SAVED_IMAGES_PATH + "_".join([str(epoch), str(steps)]) + ".jpg"
        cv2.imwrite(image_path, image)

## Train Model

In [ ]:
def train():
    start_time = datetime.now()
    saved_time = start_time
    
    for epoch in range(EPOCHS):
        steps = 1
        while True:
            
            imgs_A, imgs_B = data.get_data(BATCH_SIZE)
            
            # If value is None it means it is out of data,
            # it auto resets the data loader and breaks out of 'while' loop and starts as next batch
            if imgs_A is None or imgs_B is None:
                break
                        
            # assign local batch_size 
            batch_size = imgs_A.shape[0]
            
            # Rescale the image value from 255 => -1 to 1
            imgs_A = imgs_A / 127.5 - 1
            imgs_B = imgs_B / 127.5 - 1
            
            # Discriminator Ground Truth
            real = np.ones((batch_size,) + patch)
            fake = np.zeros((batch_size,) + patch)
            
            # Train Discriminator
            fake_A = GEN_BA.predict(imgs_B)
            fake_B = GEN_AB.predict(imgs_A)
            
            dA_loss_real = DCRM_A.train_on_batch(imgs_A, real)
            dA_loss_fake = DCRM_A.train_on_batch(fake_A, fake)
            dA_loss = 0.5 * np.add(dA_loss_real, dA_loss_fake)
            
            dB_loss_real = DCRM_B.train_on_batch(imgs_B, real)
            dB_loss_fake = DCRM_B.train_on_batch(fake_B, fake)
            dB_loss = 0.5 * np.add(dB_loss_real, dB_loss_fake)
            
            d_loss = 0.5 * np.add(dA_loss, dB_loss)
            
            
            # Train Generator
            # Training the Generator twice as to catch up with the discriminator
            for i in range(2):
                g_loss = COMBINED.train_on_batch([imgs_A, imgs_B],
                                                 [real, real,
                                                  imgs_A, imgs_B,
                                                  imgs_A, imgs_B])

            
            # Save Model
            current_time = datetime.now()
            difference_time = current_time - saved_time
            if difference_time.seconds >= (TIME_INTERVALS * 60):
                save_model()
                save_image(epoch, steps)
                saved_time = current_time
                
            # Print and Save Log
            log = "Ep: %d, steps: %d, D loss: %f, acc: %3d%%, G loss: %f" %(epoch,
                                                                                steps, d_loss[0], 100*d_loss[1],
                                                                                g_loss[0])
            print(log)
            save_log(log)
            steps += 1

In [ ]:
## Loads model if exist in Checkpoint Path
load_model()

Models not Found


In [ ]:
%%time
train()

Instructions for updating:
Use tf.cast instead.


C:\Users\Valken\Anaconda3\envs\CondaPy\lib\site-packages\keras\engine\training.py:490: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'


Ep: 0, steps: 1, D loss: 0.434775, acc:  37%, G loss: 11.257547
Ep: 0, steps: 2, D loss: 0.384252, acc:  48%, G loss: 10.026544
Ep: 0, steps: 3, D loss: 0.344495, acc:  52%, G loss: 9.773030
Ep: 0, steps: 4, D loss: 0.388114, acc:  44%, G loss: 8.691401
Ep: 0, steps: 5, D loss: 0.487126, acc:  29%, G loss: 7.631575
Ep: 0, steps: 6, D loss: 0.378985, acc:  41%, G loss: 9.061798
Ep: 0, steps: 7, D loss: 0.091126, acc:  86%, G loss: 8.535134
Ep: 0, steps: 8, D loss: 0.209564, acc:  71%, G loss: 7.837389
Ep: 0, steps: 9, D loss: 0.437955, acc:  46%, G loss: 7.083963
Ep: 0, steps: 10, D loss: 0.449825, acc:  44%, G loss: 7.062690
Ep: 0, steps: 11, D loss: 0.466687, acc:  37%, G loss: 6.377492
Ep: 0, steps: 12, D loss: 0.275514, acc:  63%, G loss: 6.731289
Ep: 0, steps: 13, D loss: 0.226670, acc:  69%, G loss: 6.317957
Ep: 0, steps: 14, D loss: 0.218672, acc:  69%, G loss: 5.964538
Ep: 0, steps: 15, D loss: 0.339007, acc:  42%, G loss: 4.622829
Ep: 0, steps: 16, D loss: 0.338979, acc:  51%, 

Ep: 5, steps: 5, D loss: 0.282232, acc:  51%, G loss: 3.244949
Ep: 5, steps: 6, D loss: 0.253156, acc:  55%, G loss: 3.490133
Ep: 5, steps: 7, D loss: 0.133808, acc:  85%, G loss: 3.402808
Ep: 5, steps: 8, D loss: 0.202050, acc:  72%, G loss: 3.742985
Ep: 5, steps: 9, D loss: 0.323136, acc:  41%, G loss: 3.022624
Ep: 5, steps: 10, D loss: 0.289987, acc:  46%, G loss: 3.272429
Ep: 5, steps: 11, D loss: 0.326551, acc:  33%, G loss: 3.438382
Ep: 5, steps: 12, D loss: 0.249186, acc:  55%, G loss: 3.261439
Ep: 5, steps: 13, D loss: 0.238997, acc:  56%, G loss: 3.061539
Ep: 5, steps: 14, D loss: 0.224387, acc:  64%, G loss: 2.956791
Ep: 5, steps: 15, D loss: 0.268010, acc:  49%, G loss: 2.820446
Ep: 5, steps: 16, D loss: 0.292066, acc:  51%, G loss: 3.175693
Ep: 5, steps: 17, D loss: 0.254453, acc:  55%, G loss: 3.271388
Ep: 5, steps: 18, D loss: 0.259094, acc:  55%, G loss: 3.402173
Ep: 5, steps: 19, D loss: 0.329985, acc:  34%, G loss: 2.989421
Ep: 5, steps: 20, D loss: 0.237872, acc:  65%

Ep: 10, steps: 8, D loss: 0.193926, acc:  73%, G loss: 3.635544
Ep: 10, steps: 9, D loss: 0.318287, acc:  35%, G loss: 2.799087
Ep: 10, steps: 10, D loss: 0.268581, acc:  53%, G loss: 2.846242
Ep: 10, steps: 11, D loss: 0.319192, acc:  32%, G loss: 2.994699
Ep: 10, steps: 12, D loss: 0.229746, acc:  63%, G loss: 2.797205
Ep: 10, steps: 13, D loss: 0.221946, acc:  64%, G loss: 2.657612
Ep: 10, steps: 14, D loss: 0.203584, acc:  71%, G loss: 2.772214
Ep: 10, steps: 15, D loss: 0.248767, acc:  58%, G loss: 2.728554
Ep: 10, steps: 16, D loss: 0.285210, acc:  50%, G loss: 2.925973
Ep: 10, steps: 17, D loss: 0.225134, acc:  65%, G loss: 3.005263
Ep: 10, steps: 18, D loss: 0.269339, acc:  50%, G loss: 3.124548
Ep: 10, steps: 19, D loss: 0.332341, acc:  31%, G loss: 2.705921
Ep: 10, steps: 20, D loss: 0.222611, acc:  69%, G loss: 2.867001
Ep: 10, steps: 21, D loss: 0.204522, acc:  70%, G loss: 2.455363
Ep: 10, steps: 22, D loss: 0.221306, acc:  64%, G loss: 2.707001
Ep: 10, steps: 23, D loss: 

...

Ep: 806, steps: 19, D loss: 0.216577, acc:  66%, G loss: 1.557146
Ep: 806, steps: 20, D loss: 0.175500, acc:  82%, G loss: 1.641665
Ep: 806, steps: 21, D loss: 0.259863, acc:  43%, G loss: 1.435446
Ep: 806, steps: 22, D loss: 0.211668, acc:  63%, G loss: 1.659855
Ep: 806, steps: 23, D loss: 0.205567, acc:  72%, G loss: 1.831649
Ep: 806, steps: 24, D loss: 0.196122, acc:  75%, G loss: 1.548195
Ep: 806, steps: 25, D loss: 0.250712, acc:  55%, G loss: 1.684241
Data exhausted, Re Initialize
Ep: 807, steps: 1, D loss: 0.243237, acc:  57%, G loss: 1.699516
Ep: 807, steps: 2, D loss: 0.249507, acc:  53%, G loss: 1.485191
Ep: 807, steps: 3, D loss: 0.180519, acc:  81%, G loss: 1.847667
Ep: 807, steps: 4, D loss: 0.174914, acc:  84%, G loss: 1.636625
Ep: 807, steps: 5, D loss: 0.272301, acc:  54%, G loss: 1.569057
Ep: 807, steps: 6, D loss: 0.245412, acc:  52%, G loss: 1.523814
Ep: 807, steps: 7, D loss: 0.301022, acc:  35%, G loss: 1.468928
Ep: 807, steps: 8, D loss: 0.211959, acc:  68%, G los

Ep: 811, steps: 19, D loss: 0.221729, acc:  65%, G loss: 1.532903
Ep: 811, steps: 20, D loss: 0.187892, acc:  78%, G loss: 1.661834
Ep: 811, steps: 21, D loss: 0.272820, acc:  39%, G loss: 1.364182
Ep: 811, steps: 22, D loss: 0.225839, acc:  60%, G loss: 1.496234
Ep: 811, steps: 23, D loss: 0.224739, acc:  64%, G loss: 1.718374
Ep: 811, steps: 24, D loss: 0.223516, acc:  63%, G loss: 1.527890
Ep: 811, steps: 25, D loss: 0.248346, acc:  56%, G loss: 1.511591
Data exhausted, Re Initialize
Ep: 812, steps: 1, D loss: 0.229517, acc:  62%, G loss: 1.681688
Ep: 812, steps: 2, D loss: 0.241007, acc:  56%, G loss: 1.490655
Ep: 812, steps: 3, D loss: 0.186486, acc:  77%, G loss: 1.877772
Ep: 812, steps: 4, D loss: 0.181914, acc:  82%, G loss: 1.676449
Ep: 812, steps: 5, D loss: 0.269439, acc:  49%, G loss: 1.607704
Ep: 812, steps: 6, D loss: 0.249947, acc:  52%, G loss: 1.497227
Ep: 812, steps: 7, D loss: 0.306117, acc:  33%, G loss: 1.410412
Ep: 812, steps: 8, D loss: 0.225693, acc:  64%, G los

Ep: 816, steps: 18, D loss: 0.226452, acc:  64%, G loss: 1.559893
Ep: 816, steps: 19, D loss: 0.185452, acc:  81%, G loss: 1.649949
Ep: 816, steps: 20, D loss: 0.258687, acc:  44%, G loss: 1.407829
Ep: 816, steps: 21, D loss: 0.246992, acc:  56%, G loss: 1.538416
Ep: 816, steps: 22, D loss: 0.240067, acc:  56%, G loss: 1.705549
Ep: 816, steps: 23, D loss: 0.218338, acc:  66%, G loss: 1.510534
Ep: 816, steps: 24, D loss: 0.256631, acc:  55%, G loss: 1.662122
Data exhausted, Re Initialize
Ep: 817, steps: 1, D loss: 0.230878, acc:  64%, G loss: 1.738914
Ep: 817, steps: 2, D loss: 0.241961, acc:  56%, G loss: 1.511713
Ep: 817, steps: 3, D loss: 0.190764, acc:  76%, G loss: 1.866020
Ep: 817, steps: 4, D loss: 0.188238, acc:  79%, G loss: 1.687024
Ep: 817, steps: 5, D loss: 0.254225, acc:  55%, G loss: 1.624088
Ep: 817, steps: 6, D loss: 0.251603, acc:  53%, G loss: 1.500525
Ep: 817, steps: 7, D loss: 0.293803, acc:  38%, G loss: 1.369002
Ep: 817, steps: 8, D loss: 0.216937, acc:  67%, G los

Ep: 821, steps: 18, D loss: 0.228078, acc:  62%, G loss: 1.550723
Ep: 821, steps: 19, D loss: 0.189342, acc:  75%, G loss: 1.599728
Ep: 821, steps: 20, D loss: 0.268743, acc:  40%, G loss: 1.497167
Ep: 821, steps: 21, D loss: 0.214661, acc:  63%, G loss: 1.639909
Ep: 821, steps: 22, D loss: 0.215845, acc:  67%, G loss: 1.737989
Ep: 821, steps: 23, D loss: 0.219374, acc:  66%, G loss: 1.475814
Ep: 821, steps: 24, D loss: 0.254140, acc:  53%, G loss: 1.474870
Data exhausted, Re Initialize
Ep: 822, steps: 1, D loss: 0.222555, acc:  64%, G loss: 1.727718
Ep: 822, steps: 2, D loss: 0.247052, acc:  55%, G loss: 1.529938
Ep: 822, steps: 3, D loss: 0.202860, acc:  75%, G loss: 1.857432
Ep: 822, steps: 4, D loss: 0.191234, acc:  78%, G loss: 1.674299
Ep: 822, steps: 5, D loss: 0.285564, acc:  47%, G loss: 1.580270
Ep: 822, steps: 6, D loss: 0.235471, acc:  55%, G loss: 1.469123
Ep: 822, steps: 7, D loss: 0.313749, acc:  29%, G loss: 1.404095
Ep: 822, steps: 8, D loss: 0.221653, acc:  67%, G los

Ep: 826, steps: 18, D loss: 0.236159, acc:  59%, G loss: 1.583939
Ep: 826, steps: 19, D loss: 0.210048, acc:  70%, G loss: 1.555057
Ep: 826, steps: 20, D loss: 0.190792, acc:  77%, G loss: 1.656888
Ep: 826, steps: 21, D loss: 0.271275, acc:  38%, G loss: 1.457822
Saved Model
Ep: 826, steps: 22, D loss: 0.208793, acc:  66%, G loss: 1.547807
Ep: 826, steps: 23, D loss: 0.217884, acc:  65%, G loss: 1.485199
Ep: 826, steps: 24, D loss: 0.245607, acc:  56%, G loss: 1.514719
Data exhausted, Re Initialize
Ep: 827, steps: 1, D loss: 0.255986, acc:  53%, G loss: 1.591203
Ep: 827, steps: 2, D loss: 0.236712, acc:  58%, G loss: 1.465024
Ep: 827, steps: 3, D loss: 0.194822, acc:  74%, G loss: 1.786227
Ep: 827, steps: 4, D loss: 0.196528, acc:  78%, G loss: 1.611392
Ep: 827, steps: 5, D loss: 0.245532, acc:  58%, G loss: 1.642294
Ep: 827, steps: 6, D loss: 0.236273, acc:  56%, G loss: 1.505283
Ep: 827, steps: 7, D loss: 0.293513, acc:  38%, G loss: 1.479456
Ep: 827, steps: 8, D loss: 0.225095, acc: